In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
 import nltk
 
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vincentzhao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Stopwords are words that don't add influential sentimental meaning, so we remove them from our text in the tweets

In [8]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Processing

In [15]:
twitter_data = pd.read_csv('data/training.1600000.processed.noemoticon.csv', names=["target", "ids", "date", "flag", "user", "text"])

In [16]:
twitter_data.columns

Index(['target', 'ids', 'date', 'flag', 'user', 'text'], dtype='object')

In [17]:
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


convert positive and negative from 0,4 to -1,1

In [129]:
twitter_data['target'] = (twitter_data['target'] - 2)/2

Stemming

In [20]:
port_stem = PorterStemmer()

In [21]:
stop = stopwords.words("english") 

In [25]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content) # remove non alphabetical characters
    stemmed_content = stemmed_content.lower() # convert all to lower
    stemmed_content = stemmed_content.split() # convert to word list
    
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stop] # stem all words except those that are stopwords
    
    return ' '.join(stemmed_content) # return reconstructed list as a string

In [26]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [27]:
twitter_data.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


vectorize text data

In [112]:
X = twitter_data['stemmed_content']

In [113]:
print(X.shape)

(1600000,)


seperate into training and testing data

In [130]:
X_train, X_test, Y_train, Y_test = train_test_split(X, twitter_data['target'], test_size=0.1, stratify=twitter_data['target'], random_state=2)

In [131]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [116]:
print(X_train.shape)
print(X_test.shape)

(1440000, 497858)
(160000, 497858)


creating model

In [132]:
model = LogisticRegression(max_iter=1000)

In [133]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

model accuracy

In [134]:
X_train_prediction = model.predict(X_train)

training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print(training_data_accuracy)

0.8098729166666667


In [135]:
X_test_prediction = model.predict(X_test)

testing_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print(testing_data_accuracy)

0.77943125


saving model

In [136]:
import pickle

In [137]:
filename = '../models/sentiment_analysis_model.sav'

pickle.dump(model, open(filename, 'wb'))

In [138]:
filename = '../models/vectorizer.sav'

pickle.dump(vectorizer, open(filename, 'wb'))

In [139]:
pred = model.predict(X_train[0])

print(pred, Y_train[0])

[-1.] -1.0


In [148]:
tweet = "I #renenberwhen i got attacked by a dog walkin 2 da bus stop.....i got on da bus and i didnt have no lunch money.i got robbed by a dog...SMH"

tweet = stemming(tweet)

vectorized = vectorizer.transform([tweet])

print(model.predict_proba(vectorized))

[[0.92176527 0.07823473]]
